In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/raw/train.csv")

In [6]:
# --- 1. Đếm số lượng dòng có MasVnrType null VÀ MasVnrArea = 0 ---
condition_null_type_zero_area = (df['MasVnrType'].isnull()) & (df['MasVnrArea'] == 0)
count_null_type_zero_area = df[condition_null_type_zero_area].shape[0]

# --- 2. Đếm số lượng dòng có MasVnrType null VÀ MasVnrArea khác 0 ---
condition_null_type_nonzero_area = (df['MasVnrType'].isnull()) & (df['MasVnrArea'] != 0)
count_null_type_nonzero_area = df[condition_null_type_nonzero_area].shape[0]

print(f"Số lượng dòng có MasVnrType null và MasVnrArea = 0: {count_null_type_zero_area}")
print(f"Số lượng dòng có MasVnrType null và MasVnrArea khác 0: {count_null_type_nonzero_area}")

Số lượng dòng có MasVnrType null và MasVnrArea = 0: 859
Số lượng dòng có MasVnrType null và MasVnrArea khác 0: 13


In [ ]:
## Có nghĩa số lượng thực sự missing là 13 dòng thôi

In [11]:
# Tính tương quan Pearson giữa MasVnrArea và cột YearBuilt và OverallQual và exterqual
correlation = df['MasVnrArea'].corr(df['YearBuilt'], method='pearson')
correlation_qual = df['MasVnrArea'].corr(df['OverallQual'], method='pearson')
correlation_exterqual = df['MasVnrArea'].corr(df['ExterQual'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1}), method='pearson')

print(f"Tương quan Pearson giữa MasVnrArea và YearBuilt: {correlation}")
print(f"Tương quan Pearson giữa MasVnrArea và OverallQual: {correlation_qual}")
print(f"Tương quan Pearson giữa MasVnrArea và ExterQual: {correlation_exterqual}")


Tương quan Pearson giữa MasVnrArea và YearBuilt: 0.3157070624361977
Tương quan Pearson giữa MasVnrArea và OverallQual: 0.41187566727768404
Tương quan Pearson giữa MasVnrArea và ExterQual: 0.3515235110463837


In [17]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

def kmeans_impute_masvnr(df):
    """
    Điền giá trị thiếu (NaN) cho MasVnrArea (Mean) và MasVnrType (Mode) 
    bằng K-Means Imputation, sử dụng OverallQual, YearBuilt, và ExterQual 
    làm các đặc trưng phân nhóm.
    """
    df_processed = df.copy()

    # 1. Mã hóa ExterQual sang số
    exter_qual_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
    df_processed['ExterQual_Num'] = df_processed['ExterQual'].map(exter_qual_mapping)

    features = ['OverallQual', 'YearBuilt', 'ExterQual_Num']
    target_area = 'MasVnrArea'
    target_type = 'MasVnrType'

    # --- 2. K-Means Clustering: Fit trên tập dữ liệu NON-MISSING của MasVnrArea ---
    
    missing_area_mask = df_processed[target_area].isnull()
    df_non_missing_area = df_processed[~missing_area_mask].copy()

    # Scaling Features
    scaler = StandardScaler()
    X_non_missing_area_scaled = scaler.fit_transform(df_non_missing_area[features])

    # Fit K-Means
    K = 4 
    kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
    df_non_missing_area['Cluster'] = kmeans.fit_predict(X_non_missing_area_scaled)

    # --- 3. Imputation cho MasVnrArea (8 NaNs) ---

    cluster_means_area = df_non_missing_area.groupby('Cluster')[target_area].mean()
    df_missing_area = df_processed[missing_area_mask].copy()
    X_missing_area_scaled = scaler.transform(df_missing_area[features])
    df_missing_area['Cluster'] = kmeans.predict(X_missing_area_scaled)
    
    # Điền giá trị MasVnrArea
    for cluster_id in range(K):
        mean_area = cluster_means_area.get(cluster_id, 0) 
        df_missing_area.loc[df_missing_area['Cluster'] == cluster_id, target_area] = mean_area
    df_processed.loc[missing_area_mask, target_area] = df_missing_area[target_area]

    # --- 4. Imputation cho MasVnrType (13 NaNs) ---
    
    missing_type_mask = df_processed[target_type].isnull()
    df_missing_type = df_processed[missing_type_mask].copy()

    # Tính Mode MasVnrType cho mỗi Cluster
    def get_mode(series):
        mode = series.mode()
        return mode.iloc[0] if not mode.empty else 'None'

    cluster_modes_type = df_non_missing_area.groupby('Cluster')[target_type].agg(get_mode)
    
    # Predict Cluster cho 13 dòng thiếu MasVnrType
    X_missing_type_scaled = scaler.transform(df_missing_type[features])
    df_missing_type['Cluster'] = kmeans.predict(X_missing_type_scaled)
    
    # Điền giá trị MasVnrType
    for cluster_id in range(K):
        mode_type = cluster_modes_type.get(cluster_id, 'None')
        df_missing_type.loc[df_missing_type['Cluster'] == cluster_id, target_type] = mode_type
        
    df_processed.loc[missing_type_mask, target_type] = df_missing_type[target_type]
    
    # --- 5. Tổng hợp dữ liệu và Kiểm tra ---
    df_processed.drop(columns=['ExterQual_Num'], inplace=True, errors='ignore')
    
    return df_processed

In [20]:
df_imputation = kmeans_impute_masvnrarea(df)
df_imputation[['MasVnrArea', 'MasVnrType']].isnull().sum()


MasVnrArea      0
MasVnrType    872
dtype: int64